<a href="https://colab.research.google.com/github/dvircohen0/Machine-Learning-Algorithms-From-Scratch/blob/main/Adaboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [14]:
from google.colab import files 
files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d ronitf/heart-disease-uci
!unzip "/content/heart-disease-uci.zip"

Saving kaggle.json to kaggle (2).json
heart-disease-uci.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  /content/heart-disease-uci.zip
  inflating: heart.csv               


In [155]:
data = pd.read_csv('/content/heart.csv')
target= data['target']
data =data.drop(['target'], axis=1)
data = preprocessing.scale(data)
target[target==0] = -1
x_train,x_test,y_train,y_test = train_test_split(data,target) 

In [156]:
class descion_tree:
  def __init__(self):
    self.sign = 1
    self.feature_ind = None
    self.treshold =None
    self.alpha = None

  def predict(self,X):
    n_samples = X.shape[0]
    X_feature = X[:,self.feature_ind]
    prediction = np.ones(n_samples)
    if self.sign == 1:
      prediction[X_feature < self.treshold] = -1
    else: prediction[X_feature > self.treshold] = -1
    return prediction


In [157]:
class Adaboost:
  def __init__(self, n_clf=5):
    self.n_clf=n_clf


  def fit(self, X, y):
    n_samples, n_features = X.shape

    W = np.full(n_samples , 1/n_samples)

    self.clfs = []
    for _ in range(self.n_clf):
      clf = descion_tree()

      min_error = float("inf")
      for i_feature in range(n_features):
        X_column = X[:,i_feature]
        tresholds = np.unique(X_column)

        for treshold in tresholds:
          p = 1
          predictions= np.ones(n_samples)
          predictions[X_column < treshold] = -1
          missclassified = W[y!= predictions]
          error = sum(missclassified)

          if error > 0.5:
            error = 1 - error
            p = -1

          if error < min_error:
            min_error = error
            clf.sign = p
            clf.treshold = treshold
            clf.feature_ind = i_feature

      EPS = 1e-10
      clf.alpha = 0.5 * np.log((1-error)/ (error + EPS))
      predictions = clf.predict(X)
      W *= np.exp(-clf.alpha * y * predictions)
      W /= np.sum(W)
      self.clfs.append(clf)


  def predict(self,X):
    clf_preds = [clf.alpha * clf.predict(X) for clf in self.clfs]
    y_pred = np.sum(clf_preds, axis= 0)
    y_pred = np.sign(y_pred)
    return y_pred


  def score(self,X,y):
    y_pred= self.predict(X)
    accuracy = np.sum(y==y_pred) / len(y)
    return accuracy


In [160]:
Adaboost_clf = Adaboost(n_clf=20)
Adaboost_clf.fit(x_train, y_train)
print('accuracy score {:.2%}'.format(Adaboost_clf.score(x_test,y_test)))

accuracy score 73.68%


In [161]:
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier(n_estimators=20)
clf.fit(x_train, y_train)
print('sklearn score {:.2%}'.format(clf.score(x_test,y_test)))

sklearn score 73.68%
